In [1]:
pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info = True, as_supervised=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


#scaling the data 
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale) 

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000 

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

#making batches

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


In [4]:
#Outlining the model

input_size = 784
output_size = 10
hidden_layer_size = 170
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
# for improvement might try different size of a hidden layer, and different sizes for different ones 
# our data (from tfds) is such that each input is 28x28x1 
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

In [5]:
#choosing the optimizer and loss fucntion

model.compile(optimizer='adam', loss ='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
#training

NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data =(validation_inputs, validation_targets), verbose=2)


Epoch 1/5
540/540 - 6s - loss: 0.2740 - accuracy: 0.9197 - val_loss: 0.1317 - val_accuracy: 0.9588
Epoch 2/5
540/540 - 3s - loss: 0.1065 - accuracy: 0.9677 - val_loss: 0.0892 - val_accuracy: 0.9718
Epoch 3/5
540/540 - 3s - loss: 0.0710 - accuracy: 0.9781 - val_loss: 0.0615 - val_accuracy: 0.9810
Epoch 4/5
540/540 - 3s - loss: 0.0516 - accuracy: 0.9836 - val_loss: 0.0489 - val_accuracy: 0.9847
Epoch 5/5
540/540 - 3s - loss: 0.0396 - accuracy: 0.9877 - val_loss: 0.0440 - val_accuracy: 0.9863


In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 315ms/step - loss: 0.0835 - accuracy: 0.9750


In [10]:
print ('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100.))

Test loss: 0.08. Test accuracy: 97.50%
